In [1]:
from imports import *

In [82]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
openai_token = os.environ['OPENAI_API_KEY']

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)



openai_ef = embedding_functions.OpenAIEmbeddingFunction(
            api_key=openai_token,
            model_name="text-embedding-3-large"  # Choose your embedding model
        )
chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Use an in-memory or persistent database
#self.collection = chroma_client.get_or_create_collection(name="openai_embeddings", embedding_function=openai_ef)
chroma_client.delete_collection(name="openai_embeddings")
collection = chroma_client.create_collection(name="openai_embeddings", embedding_function=openai_ef)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [19]:
client = OpenAI()

In [20]:
data = pd.read_csv("youtube_titles.csv")


In [21]:
train, test = train_test_split(data, test_size=0.2)

In [22]:
ids = [f"doc_{i}" for i in range(len(train))]
documents = [train['title'].iloc[i] for i in range(len(train))]
metadatas = [{"category" : train['category_1'].iloc[i],"Vid_id" : train['vid_id'].iloc[i]} for i in range(len(train))]
vectors = [
    client.embeddings.create(
        input = train['title'].iloc[i],
        model = "text-embedding-3-large",
    ) for i in range(len(train))
           ]

In [80]:
vector_values = [vector.data[0].embedding for vector in vectors]

In [27]:
 data = {
            "ids": ids,
            "documents": documents,
            "metadata": metadatas,
            "vectors": vector_values  # Use the extracted embeddings
        }


In [28]:
        with open("output.json", "w") as f:
            json.dump(data, f, indent=4)

In [81]:
collection.add(ids = ids, documents = documents, metadatas = metadatas, embeddings = vector_values)

KeyboardInterrupt: 

In [57]:
def find_similars(product):
    vector_item = client.embeddings.create(input=product, model='text-embedding-3-large')
    results = collection.query(query_embeddings= vector_item.data[0].embedding, n_results=5)
    title = []
    vid_id = []
    for i in range(len(results['documents'][0])):
        title.append(results['documents'][0][i])
        vid_id.append(results['metadatas'][0][i]['Vid_id'])
    return title, vid_id

In [79]:
title, vid_id = find_similars("NLP")

InvalidCollectionException: Collection 9ec4b11c-91f6-4fb1-a4b8-733e607133d9 does not exist.

In [59]:
title

['Artificial Intelligence',
 'LEA',
 'Nitrogen Propelled Rocket Launcher (N.P.R.L.)',
 'Artificial Neural Networks and Deep Learning | Two Minute Papers #3',
 "It's Not Magic. It's Data - Artificial Intelligence for Business"]

In [60]:
vid_id

['5J5bDQHQR1g', 'uTDyJzsJRL4', 'jrRmID81wrc', 'rCWTOOgVXyE', 'x-zfsLuZukM']

In [71]:
def messages_for(item):
    system_message = "You estimate which video below is suitable for the keyword that user want.Only return video title and Video ID"
    title, vid_id = find_similars(item)
    user_prompt = "Here is 5 choices that you can choose from:\n"
    for i in range(len(title)):
        user_prompt += f"1. Title: {title[i]}\n Video ID: {vid_id[i]}\n"
    user_prompt += "And now the question for you:\n\n"
    user_prompt +=  f"Analyze and determine three titles suitable for the keyword: {item}\n"
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Title is , video id is"}
    ]

In [72]:
def reply_from_chat_bot(item):
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = messages_for(item)
    )
    result_from_ai = response.choices[0].message.content
    return result_from_ai

In [74]:
str1 = reply_from_chat_bot("NLP")

In [77]:
arr = str1.split("\n\n")

In [78]:
arr[0]

'Title: Artificial Intelligence\nVideo ID: 5J5bDQHQR1g'

In [83]:
with open("output.json", "r") as f:
    data = json.load(f)

In [84]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Use an in-memory or persistent database
#self.collection = chroma_client.get_or_create_collection(name="openai_embeddings", embedding_function=openai_ef)
chroma_client.delete_collection(name="openai_embeddings")
collection = chroma_client.create_collection(name="openai_embeddings", embedding_function=openai_ef)

In [85]:
ids = data.get('ids')
documents = data.get('documents')
metadata = data.get('metadata')
vectors = data.get('vectors')

In [89]:
collection.add(
            ids=ids,
            documents=documents,
            metadatas=metadata,
            embeddings=vectors
        )


In [90]:
reply_from_chat_bot("NLP")

"Video 1: Artificial Intelligence\nVideo ID: 5J5bDQHQR1g\n\nVideo 4: Artificial Neural Networks and Deep Learning | Two Minute Papers #3\nVideo ID: rCWTOOgVXyE\n\nVideo 5: It's Not Magic. It's Data - Artificial Intelligence for Business\nVideo ID: x-zfsLuZukM"